# Download the 3D BAG for your area

In this tutorial we explore how to download CityJSON files from the 3D BAG for
your area of interest (AOI).
For this you'll need your AOI polygon as WKT. You can draw the polygon in QGIS, then
get it's WKT with the *Get WKT* plugin, or via any other method.

In [1]:
import json
from pathlib import Path
import csv
import gzip
import urllib

from shapely import wkt
from owslib.wfs import WebFeatureService
from cjio import cityjson
from shapely.geometry import Polygon as ShapelyPolygon
from shapely.ops import unary_union

Load our AOI from a CSV file, into a Shapely Polygon The CRS is EPSG:28992.


In [2]:
with (Path("data") / "aoi.csv").resolve().open("r") as fo:
    wktreader = csv.reader(fo)
    # Skip the header and read only the first row
    aoi_wkt, _ = [(wkt, _id) for wkt, _id in wktreader][1]

aoi_poly = wkt.loads(aoi_wkt)

We use the latest avaliable 3D BAG version, `v210908_fd2cee53`.
Additionally, we also need the URL of the 3D BAG WFS.
Finally, the tile index is served on the `bag_tiles_3k` layer, `BAG3D_v2` namespace through the WFS.

In [3]:
CITYJSON_URL = "https://data.3dbag.nl/cityjson/v210908_fd2cee53/3dbag_v210908_fd2cee53_{TID}.json.gz"
WFS_URL = "https://data.3dbag.nl/api/BAG3D_v2/wfs"
WFS_LAYER = "BAG3D_v2:bag_tiles_3k"

With OWSLib we can query the 3D BAG WFS and get a list of tile ID-s that intersect the
boundig box or our AOI.

In [4]:
def get_tile_ids(wfs_url, wfs_layer, bbox):
    wfs11 = WebFeatureService(url=wfs_url, version='1.1.0')
    response = wfs11.getfeature(typename=wfs_layer, bbox=bbox, srsname='urn:x-ogc:def:crs:EPSG:28992', outputFormat='json')

    tiles = json.loads( response.read().decode('utf-8') )['features']
    tile_ids = [ tile['properties']['tile_id'] for tile in tiles ]

    return tile_ids

In [5]:
tile_ids = get_tile_ids(WFS_URL, WFS_LAYER, aoi_poly.bounds)

Once we have the tile IDs, download the cityjson files.

In [6]:
def download_3dbag(tile_ids, tilesdir):
    fnames = []
    for tid in tile_ids:
        url = CITYJSON_URL.format(TID=tid)
        print(url)
        fname = tilesdir / (tid + '.json')
        try:
            with urllib.request.urlopen(url) as response, open(fname, 'wb') as out_file:
                data = response.read()  # a `bytes` object
                out_file.write(gzip.decompress(data))
                fnames.append(fname)
        except urllib.error.HTTPError as err:
            print(err)

    return fnames

In [7]:
filenames = download_3dbag(tile_ids, Path("data").resolve())

https://data.3dbag.nl/cityjson/v210908_fd2cee53/3dbag_v210908_fd2cee53_5785.json.gz
https://data.3dbag.nl/cityjson/v210908_fd2cee53/3dbag_v210908_fd2cee53_5786.json.gz


Load the files from `filenames` and merge them into a single model.

In [8]:
# Take the first model as base
cm = cityjson.load(filenames[0])
print(f"Loaded a citymodel with {len(cm.cityobjects)} CityObjects")
# Read the rest
cms = []
for p in filenames[1:]:
    _cm = cityjson.load(p, transform=True)
    cms.append(_cm)
    print(f"Loaded a citymodel with {len(_cm.cityobjects)} CityObjects")
# Merge
cm.merge(cms)

Loaded a citymodel with 2265 CityObjects
Loaded a citymodel with 2622 CityObjects


True

The method `load_from_j` is required after using the CLI methods via the API,
because the CLI methods only affect the `j` member of the CityJSON object.
So the contents of `j` need to be loaded back to the API members.

In [9]:
cm.load_from_j()

Verify that we have the correct number of CityObjects.

In [10]:
print(f"Nr. of cityobjects: {len(cm.cityobjects)}")

Nr. of cityobjects: 4887


The 3D BAG contains the building footprints as LoD0 models. These models have the type
`Building`, instead of the `BuildingPart`-s that store the 3D models. Using these
footprints, we can intersect with our AOI to create our final subset. Shapely helps
with calculating the intersection.

The LoD0 models (footprints) are stored as `MultiSurface` in the CityJSON, so we need
to extract the ring from within the multisurface to be able to create a Shapely Polygon.

In [11]:
aoi_cm = cityjson.CityJSON()

buildings = cm.get_cityobjects(type="building")
for co_id, co in buildings.items():
    for geom in co.geometry:
        if str(geom.lod) == "0":
            fp_poly = unary_union([ShapelyPolygon(ring)
                                   for surface in geom.boundaries
                                   for ring in surface])
            if fp_poly.intersects(aoi_poly):
                aoi_cm.set_cityobjects({co_id: co})
                # also add its children
                if len(co.children) > 0:
                    ch = cm.get_cityobjects(id=co.children)
                    aoi_cm.set_cityobjects(ch)

Since we created a complete new citymodel, it's good to update some of the metadata
in the model.

For this we need the CLI methods, and thus we need to update the `j` member on the
citymodel.

In [14]:
aoi_cm.add_to_j()

aoi_cm.update_bbox()
aoi_cm.set_epsg(7415)

True

In [15]:
cityjson.save(aoi_cm, "outfile.json")

## Explore further

+ Instead of writing and reading from files, merge the downloaded citymodels in memory.